# Programación Entera: Heurísticas para el Problema de Asignación

#### Descripción

En este trabajo se estudia el problema de asignación de manera detallada, en concreto el de asignación clásico y el de asignación generalizado, analizando el problema de programación lineal además de heurísticas greedy y de búsqueda local. Todo ello se ha hecho apoyandose en el lenguaje *R* y el solver *glpk*. Por último se ha comprobado la validez de los métodos sobre distintos problemas a modo de ejemplo.

Este documento forma parte de un grupo de trabajos relacionados con la heuristicas para la resolución de problemas de programación entera. Existen otros tres trabajos referidos al estudio del [problema de la mochila](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-knapsack-heuristics.ipynb), a [problemas de redes](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-network-heuristics.ipynb) y a [problemas de localización de servicios](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-service-location-heuristics.ipynb). El contenido completo del trabajo está publicado en el siguiente repositorio: https://github.com/garciparedes/integer-programming-heuristics 

#### Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
#### Fecha

  * Mayo de 2018


## Contenidos
  
  * [Introducción](#Introducción)
  * [Problema de Asignación](#Problema-de-Asignación)
    * [Asignación Clásico](#Asignación-Clásico)
    * [Asignación Generalizado](#Asignación-Generalizado)
  * [Conclusiones](#Conclusiones)

## Introducción

El estudio de problemas de programación lineal binaria es un área de investigación muy interesante y enriquecedor, por su amplio campo de apliación en situaciones reales. En este caso, el trabajo se centra en el estudio de problemas de asignación, es decir, aquellos que se basan en la búsqueda de la configuración óptima en algún sentido entre un conjunto de productores y otro de consumidores. 

Estos problemas modelizan un gran número de situaciones. Entre ellas, una de las más destacadas es la asignación de tareas a un grupo de individuos de tal manera que se minimicen los costes globales de la planificación. 

La resolución de óptima de problemas de programación lineal binaria conlleva un elevado coste computacional, que en muchos casos se hace inmanejable cuando el tamaño del conjunto de datos de entrada es muy elevado. Esto es cuando hay muchas tareas que asignar a muchos individuos. Por tanto, en la literatura se han propuesto distintas heurísticas, que a pesar de no ofrecer ninguna garantía de optimalidad, en la práctica ofrecen resultados muy cercanos a dicho valor con un coste computacional mucho menor.

En este trabajo se han estudiado dos casos concretos del problema de asignación: el referido a **asignación clásico**, donde tanto productores como consumidores sólo pueden relacionarse de manera **uno a uno**, y el referido a **asignación generalizada**, donde un productor puede servir a más de un consumidor, por lo que la relación es **uno a muchos**. Para cada tipo de problema, se ha estudiado el planteamiento exacto, basado en programación lineal, además de una heurística *Greedy* y una mejora basada en búsqueda local. Por último, se ha comprobado la validez de dichos planteamientos aplicando dichas técnicas a una serie de problemas reales.

Todo el planteamiento se ha llevado a cabo apoyandose en el lenguaje de computación estadística *R*, sobre el cual se han implementado las diferentes heurísticas, además de resolver los problemas de manera exacta. Para ello se ha utilizado el *solver* de de programación mixta (lineal y entera) **glpk**.

A continuación se muestran distintas sentencias para configurar el entorno de desarrollo, así como funciones de lectura de ficheros (`ReadAssignment()` para la lectura de los conjuntos de datos referidos a problemas de asignación clásica, `ReadGeneralizedAssignment()` para los correspondientes problemas de asignación generalizada). Por último, se ha definido un "wrapper" denominado `Solve()` para la llamada al solver *glpk*.

#### Configuración del Entorno

In [1]:
rm(list = ls())

In [2]:
library(slam)
library(Rglpk)
library(magrittr)

Using the GLPK callable library version 4.65


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

#### Constantes y Funciones de Apoyo

In [4]:
dataset.folder <- "./data/assignment/"

In [5]:
ReadAssignment <- function(file.name) {
    c <- as.matrix(read.csv(file.name, header = FALSE))
    colnames(c) <- NULL
    list(
        m = dim(c)[1],
        n = dim(c)[2],
        cost = c
    ) %>%
    return()
}

In [6]:
ReadGeneralizedAssignment <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.integer()
    close(con)
    list(m        = raw[1],
         n        = raw[2], 
         cost     = matrix(raw[3:(2 + raw[1] * raw[2])], 
                           nrow = raw[1], ncol = raw[2], byrow = TRUE),
         resource = matrix(raw[(3 + raw[1] * raw[2]):(2 + 2 * raw[1] * raw[2])], 
                           nrow = raw[1], ncol = raw[2], byrow = TRUE),
         capacity = raw[(3 + 2 * raw[1] * raw[2]):length(raw)]
        
    ) %>%
    return()
}

In [7]:
Solve <- function(f.obj, f.con, f.dir, f.rhs, ...) {
    Rglpk_solve_LP(f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de Asignación

El *problema de asignación* es ampliamente conocido y ha sido estudiado en profundidad en la literatura, principalmente debido a que permite modelizar una gran cantidad de situaciones que se dan en la vida real. Por estas razones es necesario encontrar un modelo matemático que permita representar y estudiar el problema de manera analítica. 

De manera natural, el problema de asignación puede ser enunciado como el modelo que representa aquellas situaciones en que se tiene dos grupos disjuntos de elementos, tales que el primero de ellos está constituido por productores mientras que el segundo grupo se compone de consumidores. El problema se basa en asignar a cada consumidor un determinado productor, de tal manera que todos ellos queden cubiertos. Este problema puede ser ampliado de muchas maneras, tal y como se verá posteriormente, sin embargo el objetivo último es minimizar (o maximizar) una determinada función de coste (o beneficio) encontrando la configuración óptima de asignaciones. 

Tal y como se puede apreciar en el párrafo anterior, este problema puede modelizarse como de programación lineal binaria, de tal manera que cada variable represente de si un productor sirve on no a un consumidor restringido a unas determinadas restricciones de capacidad y demanda (dependiendo del modelo concreto). No es difícil darse que este problema es muy similar al [Problema de Transporte](https://es.wikipedia.org/wiki/Problema_de_transporte), tal es así que la diferencia reside en la imposición de que las variables de decisión sean de naturaleza binaria. Por tanto, el problema de transporte puede ser visto como una relajación del problema de asignación.


A continuación, se estudirán de manera detallada dos problemas enmarcados en la categoría de problemas de asignación, siendo el primero de ellos el de [Asignación Clásico](#Asignación-Clásico), y el segundo [Asignación Generalizado](#Asignación-Generalizado). Para estos problemas se va a definir el modelo de programación lineal que los representa, para posteriormente añadir el código fuente en el lenguaje *R* que permite la resolución *Exacta* de dichos problemas. Además, se indicarán las heurísticas *Greedy* junto con su correspondiente mejora de *Búsqueda Local*, todas ellas implementadas en el lenguaje *R* para comprobar su efectividad. Por último se han resuelto distintos problemas (referidos a cada modelización) para comprobar la validez de las heurísticas, así como su cercanía a la solución óptima obtenida por el método exacto.

### Asignación Clásico

#### Descripción:

El *problema de asignación clásico* se corresponde con el caso más básico posible, en el cual el número de productores y de consumidores es el mismo, y el objetivo es la búsqueda de la asignación óptima siguiendo la configuración **1 productor a 1 consumidor** de tal manera que se minimice (o maximice) una determinada función de coste.

Para ello, la modelización más sencilla es la basada en una matriz binaria de variables de decisión denominada $X$ y formada por $m$ filas que representan a los productores y $n$ columnas que representan a los consumidores. De esta manera se dice que si la posición $x_{ij}$ de la matriz contiene el valor $1$ el productor $i$ servirá al consumidor $j$, mientras que no lo hará si esta posición contiene el valor $0$. Para poder cuantificar el coste (o valor) de cada una de las posiciones de la matriz de decisiones y así encontrar la solución óptima para el problema, nos apoyaremos en una matriz de costes denotada por $C$ y con las mismas dimensiones. Las restricciones sobre las que se define este problema por tanto, pueden ser enunciadas de tal manera que cada productor sirva únicamente a un consumidor, así como que cada consumidor sea servido únicamente por un productor.

#### Modelo:

Una vez descrita la notación necesaria para poder modelizar el problema así como una descripción acerca de las restricciones necesarias para que este sea coherente con la realidad que pretender representar, el siguiente paso es indicar su formulación:

\begin{equation}
    \begin{array}{ll@{}ll}
      \text{Minimizar} & \displaystyle \sum\limits_{i=1}^{m}\sum\limits_{j=1}^{n} c_{ij} \cdot x_{ij} \\
      \text{sujeto a}  & \sum\limits_{j=1}^{n} x_{ij} = 1, & \forall i \in \{1,...,m\} \\
                       & \sum\limits_{i=1}^{m} x_{ij} = 1, & \forall j \in \{1,...,n\} \\
                       & x_{ij} \in \{0, 1\}, 	& \forall i \in \{1,...,m\}, \forall j \in \{1,...,n\}
    \end{array}
\end{equation}

Una vez definido el *problema de asignación clásico* de tal manera que se pueda construir de manera clara y bien definida una función objetivo, así como una matriz de restricciones, el siguiente paso es realizar la codificación pertinente en el lenguaje *R*, la cual se puede consultar en la función `AssignmentExact()`, que tan solo requiere de una matriz de costes para e indicar si el problema el sentido del problema (minimización o maximización) para devolver la solución óptima del problema. 

In [8]:
AssignmentExact <- function(p, ...) { 
    conditions <- matrix(0, nrow = p$n + p$m, ncol = p$n * p$m)
    for (i in 1:p$m) {
        conditions[i, seq(p$n) + (i - 1) * p$n] <- 1
    }
    for (i in (p$m + 1):(p$n + p$m)) {
        conditions[i, seq(i - p$m, p$m * p$n, by = p$n)] <- 1
    }
    Solve(p$cost, conditions, c(rep("==", p$m + p$n)), 
          rep(1, p$n + p$m),  ...)
}

#### Heurísticas:

La dificultad práctica que surge mediante la resolución exacta del problema se debe principalemente al coste computacional necesario para llegar a la solución deseada cuando el tamaño de la matriz de costes es elevado. Por tanto, existen otras técnicas conocidas como heurísticas que a pesar de no suministrar un resultado óptimo en todos los casos, generan resultados próximos a él con un coste computacinal mucho menor. Es por ello que a continuación se van a enunciar dos heurísticas que permiten resolver el problema en un tiempo mucho menor. Primeramente describiremos el funcionamiento de la heuristica *Greedy* para posteriormente realizar una mejora sobre ella con el método de *Busqueda Local*.

##### Greedy:

La heurística *Greedy* recibe ese nombre precisamente debido por su método de comportamiento. La traducción a lenguaje Castellano podría ser algo así como *hambriento* o *voraz* ya que esta nunca "retrocede" y cambia una decisión tomada. Es decir, una vez fijada una asignación entre unos determinados productores y consumidores, esta no cambia para buscar otra configuración más cercana al óptimo real. Esto hace que el la implementación algorítmica de la solución Greedy sea extremadamente eficiente, ofreciendo resultados razonables. La regla de decisión que utiliza es la elección de la asignación que más minimiza (o maximiza) la función de coste de entre todas las posibles asignaciones coherentes en esa iteracción. Esto quiere decir que si un productor ya ha sido utilizado, no tendrá sentido estudiar si se puede asignar a otro consumidor, y viceversa. 

A continaución se ilustra la implementación realizada de la heurística *Greedy*, recogida en la función `AssignmentGreedy()`, que tan solo requiere de la matriz de costes así como de la métrica a optimizar, para devolver una solución aproximada en un tiempo reducido:

In [9]:
AssignmentGreedy <- function(p) {   
    demand <- rep(1, dim(p$cost)[1])
    capacity <- rep(1, dim(p$cost)[2]) 
    sol <- list(solution = matrix(0, nrow = p$m, ncol = p$n), 
                objval = 0, slack = capacity)
    while (!all(demand == 0)) {
        ij <- which(p$cost == min(p$cost[sol$slack > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- which(sol$slack[ij[, 1]] > 0 & demand[ij[, 2]] > 0)[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(sol$slack[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        sol$slack[i] <- sol$slack[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + p$cost[i, j, drop = TRUE] * inc
    }
    return(sol)
}

##### Búsqueda Local:

La heurística de *Búsqueda Local* en realidad es una técnica para optimizar soluciones iniciales y tratar de mejorar en la medida de lo posible (un entorno local) la solución inicial. Esto se lleva a cabo de la siguiente manera: Partiendo de una solución inicial que asumimos factible, el método trata de realizar una permutación entre los consumidores de dos productores de tal manera que el coste se reduzca. Al igual que sucedía en el caso de la heurística *Greedy*, ahora también se sigue la misma filosofía, en el sentido de que en cada iteracción del método, se realiza el cambio que mayor beneficio proporciona.

La implementación de esta solución se ha dividido en tres partes, partiendo inicialmente de una solución obtenida por el método *Greedy*. El siguiente paso a realizar es definir la heurística de mejora o beneficio, la cual se ha llevado a cabo en la función `GreedyLocalSearchImprovement()`, que dados dos pares de índices (referidos dos asignaciónes (productor, consumidor)). Este beneficio se calcula de la siguiente manera: 

$$(c_{i_1j_2} + c_{i_2j_1}) - (c_{i_1j_1} + c_{i_2j_2})$$

Es decir, el beneficio será el nuevo coste de la asignación (dos primeros términos) menos el coste anterior de la asignación (dos últimos términos). Una vez calculados dichos valores, es necesario quedarse con el máximo de entre todos ellos, lo cual se lleva a cabo a partir de la función `GreedyLocalSearchBest()`. El último paso es el de realizar el cambio. Dicho cambio se realiza en la función que controla el algoritmo de búsqueda local (finaliza cuando ya no hay nuevos cambios que mejoren el la función objetivo). Esta función se ha denotado por `AssignmentGreedyLocalSearch()`.

In [10]:
GreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    (cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) -
    (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]])
}

In [11]:
GreedyLocalSearchBest <- function(cost, changes) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:(nrow(changes) - 1)) {
        for(j in (i + 1):nrow(changes)) {
            v.temp <- GreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                   changes[j, , drop = FALSE],
                                                   cost)
            if (best$v < v.temp) {
                best$v <- v.temp
                best$i <- i
                best$j <- j
            }
        }
    }
    return(best)
}

In [12]:
AssignmentGreedyLocalSearch <- function(p) {    
    sol <- AssignmentGreedy(p)
        
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    best <- GreedyLocalSearchBest(p$cost, changes)
    while(best$v > 0) {

        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$objval <- sol$objval - best$v
        
        best <- GreedyLocalSearchBest(p$cost, changes)
    }
    return(sol)
}

### Ejemplos

A continuación se va a demostrar la validez de los algoritmos implementados mediante la prueba sobre tres problemas de minimización de tamaño $15x15$ los cuales se definen a partir de la matriz de costes. Nótese que en estos casos no es necesario indicar en el procedimiento de resolución exacta que la solución dada debe ser binaria y no real, debido a la *propiedad de integridad* que asegura que si los coeficientes de un problema de transporte son enteras, entonces los valores de las variables de decisión también lo serán. Dado que el problema de asignación clásico es una caracterización del problema de transporte, esta condición también se cumplirá. 

##### matriz.15x15.1

In [13]:
matriz.15x15.1 <- ReadAssignment(paste0(dataset.folder, "matriz_15x15_1.csv"))

In [14]:
s <- AssignmentExact(matriz.15x15.1)
list(solution = matrix(s$solution, nrow = matriz.15x15.1$n), 
     objval   = s$optimum)

0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [15]:
s <- AssignmentGreedy(matriz.15x15.1)
list(solution = s$solution,
     objval   = s$objval)

0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [16]:
s <- AssignmentGreedyLocalSearch(matriz.15x15.1)
list(solution = s$solution,
     objval   = s$objval)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Tal y como se puede apreciar a la vista de los resultados, para el problema [matriz.15x15.1](#matriz.15x15.1), la solución óptima ($135$) siempre es menor que la obtenida por los métodos *Greedy* ($218$) y *Búsqueda Local* ($144$). Sin embargo, tal y como era de esperar, la solución optimizada a partir de la búsqueda local mejora de manera sustancial el resultado *Greedy*. A pesar de no llegar a tomar el valor óptimo, está muy cercano de manera relativa.

##### matriz.15x15.2

In [17]:
matriz.15x15.2 <- ReadAssignment(paste0(dataset.folder, "matriz_15x15_2.csv"))

In [18]:
s <- AssignmentExact(matriz.15x15.2)
list(solution = matrix(s$solution, nrow = matriz.15x15.2$m),
     objval   = s$optimum)

0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
s <- AssignmentGreedy(matriz.15x15.2)
list(solution = s$solution,
     objval   = s$objval)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
s <- AssignmentGreedyLocalSearch(matriz.15x15.2)
list(solution = s$solution,
     objval   = s$objval)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Tal y como se puede apreciar a la vista de los resultados, para el problema [matriz.15x15.2](#matriz.15x15.2), la solución óptima ($125$) siempre es menor que la obtenida por los métodos *Greedy* ($146$) y *Búsqueda Local* ($146$). En este caso, la optimización por búsqueda local no consigue mejorar la solución *Greedy* obtenida inicialmente. Sin embargo, esta solución se encuentra cerca del óptimo. Una situación que se da en muchos casos es que cuando las soluciones iniciales (en este caso la obtenida por el método *Greedy*) se encuentran cerca del valor óptimo, son difíciles de optimizar por métodos de búsqueda local, ya que el espacio de posibles alternativas de mejora es reducido.

##### matriz.15x15.3

In [21]:
matriz.15x15.3 <- ReadAssignment(paste0(dataset.folder, "matriz_15x15_3.csv"))

In [22]:
s <- AssignmentExact(matriz.15x15.3)
list(solution = matrix(s$solution, nrow = matriz.15x15.3$m),
     objval   = s$optimum)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [23]:
s <- AssignmentGreedy(matriz.15x15.3)
list(solution = s$solution,
     objval   = s$objval)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [24]:
s <- AssignmentGreedyLocalSearch(matriz.15x15.3)
list(solution = matrix(s$solution, nrow = matriz.15x15.3$m),
     objval  = s$objval)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Tal y como se puede apreciar a la vista de los resultados, para el problema [matriz.15x15.3](#matriz.15x15.3), la solución óptima ($153$) siempre es menor que la obtenida por los métodos *Greedy* ($205$) y *Búsqueda Local* ($156$). Sin embargo, tal y como era de esperar, la solución optimizada a partir de la búsqueda local mejora de manera sustancial el resultado *Greedy*. A pesar de no llegar a tomar el valor óptimo, está muy cercano de manera relativa.

### Asignación Generalizado

#### Descripción:

El *problema de asignación generalizado* en cierto sentido, es una ampliación respecto del descrito en la sección anterior. Esto es la modificación de una la restricción impuesta a los productores, de tal manera que estos ya no estén restringidos a la servir a un único consumidor. Esto se puede modelizar en el enfoque relacional de bases de datos como una la relación **1 productor a muchos consumidores**. Sin embargo, la restricción no se elimina por completo, sino que se fija en una determinada "capacidad" de trabajo por cada productor. Tal y como es de esperar (y para que tenga sentido la anterior afirmación), también será necesario añadir una matriz extra que refleje las unidades de capacidad necesarias por cada productor para poder poder servir a cada consumidor.

Tal y como sucedía con el problema anterior, en este caso también se cumple la idea de que el problema de asignación generalizado se puede ver como el caso del problema del transporte, para el cual las variables de decisión son de naturaleza binaria. Nótese que en este caso el modelo se encuntra "más cercano aún" ya que las capacidades no son todas ellas iguales a $1$, cosa que si sucede con las demandas.

Ahora que ya se ha ilustrado el contexto del problema, el siguiente paso es definir las variables necesarias para poder modelizarlo y que este puedas ser estudiado de manera analítica. Al igual que para el problema de asignación clásico, se tiene una matriz de decisión binaria $X$ de $m$ filas y $n$ columnas referida a las asignaciones entre el productor $i$ y el consumidor $j$. La matriz de costes $C$ a minimizar (o maximizar) también es equivalente. Las diferencias residen una nueva matriz de capacidades con las mismas dimensiones que las anteriores y que denotaremos por $R$ (del inglés *resources*) y que indica en la posición $r_{ij}$ la cantidad de recursos necesarios para que el productor $i$ pueda satisfacer al consumidor $j$. Estos deberán ser menores o iguales a la capacidad de cada productor, que almacenaremos en un vector de longitud igual al número de productores y que denotaremor por $b$.

#### Modelo:

Una vez descrita la notación pertinente, así como las modificaciones sobre el problema de asignación clásico para llegar al problema de asignación generalizado, el siguiente paso es añadir el modelo matemático de programación lineal que representa el problema. La formulación del mismo se indica a continuación:

\begin{equation}
    \begin{array}{ll@{}ll}
      \text{Minimizar} & \displaystyle \sum\limits_{i=1}^{m}\sum\limits_{j=1}^{n} c_{ij} \cdot x_{ij} \\
      \text{sujeto a}  & \sum\limits_{i=1}^{m} x_{ij} = 1, & \forall j \in \{1,...,n\} \\
                       & \sum\limits_{j=1}^{n} r_{ij} x_{ij} \leq b_{i}, & \forall i \in \{1,...,m\} \\
                       & x_{ij} \in \{0, 1\}, 	& \forall i \in \{1,...,m\}, \forall j \in \{1,...,n\}
    \end{array}
\end{equation}

Una vez definido el modelo, es sencillo realizar la implementación pertinente en el lenguaje *R*. Esto se ha llevado a cabo en la función `GeneralizedAssignmentExact()` que calcula una solución exacta y para el problema de asignación generalizado:

In [25]:
GeneralizedAssignmentExact <- function(p, ...) {
    conditions <- matrix(0, nrow = p$n + p$m, ncol = p$n * p$m)
    for (i in 1:p$m) {
        conditions[i, seq(p$n) + (i - 1) * p$n] <- p$resource[i, ]
    }
    for (i in (p$m + 1):(p$n + p$m)) {
        conditions[i, seq(i - p$m, p$m * p$n, by = p$n)] <- 1
    }
    Solve(t(p$cost), conditions, c(rep("<=", p$m), rep("==", p$n)), 
          c(p$capacity, rep(1, p$n)), types = "B", max = TRUE, ...)
}

#### Heurísticas:

Debido al elevado coste computacional necesario para la resolución de manera exacta de problemas de asignación generalizada cuando el tamaño del problema es grande, a continuación se van a enunciar dos heurísticas que permiten resolver el problema de manera aproximada en un tiempo razonable. La primera de ellas está basada en el método *Greedy* del cual se ha hablado en la anterior sección, mientras que el segundo se refiere a otra heurística de mejora basada en *Búsqueda Local*, la cual también se describió en la anterior sección.


##### Greedy de Martello-Toth:

Este método, al igual que el descrito en la sección anterior, se basa en la asinación de productores y consumidores de manera *voraz* y sin tener en cuenta decisiones pasadas. Sin embargo, en el caso anterior no era necesario tener en cuenta la capacidad restante de los consumidores para llegar a una solución factible, algo que en este caso si que lo es. Por tanto, la heurística *Greedy* que se ha utilizado se basa en la selección de la configuración que la capacidad libre total de los productores, y de entre estas, se selecciona la que más ganancia proporcione en el sentido del problema (minimizar o maximizar). Mediante la utilización de esta heurística, se asegura la obtención de una solución factible del problema. Sin embargo, las garantías de optimalidad en este caso son más difusas que en el caso del problema anterior.

La función `GeneralizedAssignmentGreedy()` implementa el cálculo de la heurística *Greedy Martello-Toth* tal y como se ha descrito en el párrafo anterior.

In [26]:
GeneralizedAssignmentGreedy <- function(p) {    
    sol <- list(solution = matrix(0, nrow = p$m, ncol = p$n), 
                objval = 0, slack = p$capacity)
    while(!all(colSums(sol$solution) == 1)) {
        max.resource <- max(sol$slack - p$resource[,colSums(sol$solution) == 0])
        
        ij <- which(sol$slack - p$resource == max.resource, arr.ind = TRUE)
        ij <- ij[colSums(sol$solution[, ij[,2], drop = FALSE]) == 0, , drop = FALSE]
        ij <- ij[which.max(p$cost[ij]), ,drop = FALSE]
        
        sol$solution[ij[1], ij[2]] <- 1
        sol$slack[ij[1]] <- sol$slack[ij[1]] - p$resource[ij[1], ij[2]]
        sol$objval <- sol$objval + p$cost[ij[1], ij[2]]
    }
    return(sol)
}

##### Búsqueda Local:

La heurística de *búsqueda local* para el problema de asignación generalizado, se basa en las mismas ideas que para el caso del problema de asignación clásico. Sin embargo, al igual que sucede con la heurística greedy, en este caso el intercambio de asignaciones debe tener en cuenta la capacidad de los productores. Esto se lleva a cabo mediante la verificación de que el nuevo intercambio cumplirá las restricciones de capacidad del productor.

La implementación de esta solución se ha dividido en tres partes, partiendo inicialmente de una solución obtenida por el método *Greedy*. El siguiente paso a realizar es definir la heurística de mejora o beneficio, la cual se ha llevado a cabo en la función `GeneralizedAssignmentGreedyLocalSearchmprovement()`, que dados dos pares de índices (referidos dos asignaciónes (productor, consumidor)). Dado que en este caso el problema de asignación calculará la mejora de beneficio de forma opuesta. Este beneficio se calcula de la siguiente manera: 

$$- (c_{i_1j_2} + c_{i_2j_1}) + (c_{i_1j_1} + c_{i_2j_2})$$

Es decir, el beneficio será el nuevo coste de la asignación (dos primeros términos) menos el coste anterior de la asignación (dos últimos términos). Una vez calculados dichos valores, es necesario quedarse con el máximo de entre todos ellos, lo cual se lleva a cabo a partir de la función `GeneralizedAssignmentGreedyLocalSearchBest()`. El último paso es el de realizar el cambio. Dicho cambio se realiza en la función que controla el algoritmo de búsqueda local (finaliza cuando ya no hay nuevos cambios que mejoren el la función objetivo). Esta función se ha denotado por `GeneralizedAssignmentGreedyLocalSearch()`.

In [27]:
GeneralizedAssignmentGreedyLocalSearchmprovement <- function(change.1, change.2, cost) {
    return( - (cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) +
              (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]]))
}

In [28]:
GeneralizedAssignmentGreedyLocalSearchBest <- function(p, changes, slack) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:(nrow(changes) - 1)) {
        for(j in (i + 1):nrow(changes)) {
            if ((slack[changes[i, , drop = FALSE][1]] + p$resource[changes[i, , drop = FALSE]] - 
                     p$resource[changes[i, 1], changes[j, 2]] > 0) & 
                (slack[changes[j, , drop = FALSE][1]] + p$resource[changes[j, , drop = FALSE]] - 
                     p$resource[changes[j, 1], changes[i, 2]] > 0)) { 
                    
                    v.temp <- GeneralizedAssignmentGreedyLocalSearchmprovement(changes[i, ,drop = FALSE], 
                                                                     changes[j, , drop = FALSE],
                                                                     p$cost)
                    if (best$v < v.temp) {
                        best$v <- v.temp
                        best$i <- i
                        best$j <- j
                    }
                }
        }
    }
    return(best)
}

In [29]:
GeneralizedAssignmentGreedyLocalSearch <- function(p) {
    sol <- GeneralizedAssignmentGreedy(p)
    
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    slack <- rowSums(sol$solution * p$resource)
    best <- GeneralizedAssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    while(best$v > 0) {
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] + p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] + p$resource[changes[best$j, , drop = FALSE]]
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] - p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] - p$resource[changes[best$j, , drop = FALSE]]
        sol$objval <- sol$objval + best$v
        best <- GeneralizedAssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    }
    return(sol)
}

### Ejemplos

Para verificar la validez de las implementaciones realizadas, en esta sección se han aplicado sobre $5$ problemas de asignación generalizada para así poder comparar los resultados obtenidos. 

##### gap2.1

In [30]:
gap2.1 <- ReadGeneralizedAssignment("./data/assignment/gap2_1.txt")

In [31]:
s <- GeneralizedAssignmentExact(gap2.1)
list(solution = matrix(s$solution, nrow = gap2.1$m, byrow = TRUE),
     objval   = s$optimum,
     slack    = s$auxiliary$primal[1:gap2.1$m])

0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0


In [32]:
GeneralizedAssignmentGreedy(gap2.1)

0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0
0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [33]:
GeneralizedAssignmentGreedyLocalSearch(gap2.1)

0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


Para el problema [gap2.1](#gap2.1) las soluciones obtenidas han sido las siguientes: el resultado exacto del problema es ($434$), el resultado obtenido por el método *Greedy* propuesto por Martello-Toth es ($377$), que tras la heurística de *Búsqueda Local* toma el valor ($432$).

##### gap4.1

In [34]:
gap4.1 <- ReadGeneralizedAssignment("./data/assignment/gap4_1.txt")

In [35]:
s <- GeneralizedAssignmentExact(gap4.1)
list(solution = matrix(s$solution, nrow = gap4.1$m, byrow = TRUE),
     objval   = s$optimum,
     slack    = s$auxiliary$primal[1:gap4.1$m])

0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0
0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [36]:
s <- GeneralizedAssignmentGreedy(gap4.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0
0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0
0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [37]:
s <- GeneralizedAssignmentGreedyLocalSearch(gap4.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0
0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1


Para el problema [gap4.1](#gap4.1) las soluciones obtenidas han sido las siguientes: el resultado exacto del problema es ($656$), el resultado obtenido por el método *Greedy* propuesto por Martello-Toth es ($541$), que tras la heurística de *Búsqueda Local* toma el valor ($650$).

##### gap6.1

In [38]:
gap6.1 <- ReadGeneralizedAssignment("./data/assignment/gap6_1.txt")

In [39]:
s <- GeneralizedAssignmentExact(gap6.1)
list(solution = matrix(s$solution, nrow = gap6.1$m, byrow = TRUE),
     objval   = s$optimum,
     slack    = s$auxiliary$primal[1:gap6.1$m])

0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [40]:
s <- GeneralizedAssignmentGreedy(gap6.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [41]:
s <- GeneralizedAssignmentGreedyLocalSearch(gap6.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Para el problema [gap6.1](#gap6.1) las soluciones obtenidas han sido las siguientes: el resultado exacto del problema es ($761$), el resultado obtenido por el método *Greedy* propuesto por Martello-Toth es ($664$), que tras la heurística de *Búsqueda Local* toma el valor ($746$).

##### gap8.1

In [42]:
gap8.1 <- ReadGeneralizedAssignment("./data/assignment/gap8_1.txt")

In [43]:
s <- GeneralizedAssignmentExact(gap8.1)
list(solution = matrix(s$solution, nrow = gap8.1$m, byrow = TRUE),
     objval   = s$optimum,
     slack    = s$auxiliary$primal[1:gap8.1$m])

1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [44]:
s <- GeneralizedAssignmentGreedy(gap8.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [45]:
s <- GeneralizedAssignmentGreedyLocalSearch(gap8.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


Para el problema [gap8.1](#gap8.1) las soluciones obtenidas han sido las siguientes: el resultado exacto del problema es ($1133$), el resultado obtenido por el método *Greedy* propuesto por Martello-Toth es ($990$), que tras la heurística de *Búsqueda Local* toma el valor ($1100$).

##### gap12.1

In [46]:
gap12.1 <- ReadGeneralizedAssignment("./data/assignment/gap12_1.txt")

In [47]:
s <- GeneralizedAssignmentExact(gap12.1)
list(solution = matrix(s$solution, nrow = gap12.1$m, byrow = TRUE),
     objval   = s$optimum,
     slack    = s$auxiliary$primal[1:gap12.1$m])

0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [48]:
s <- GeneralizedAssignmentGreedy(gap12.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
s <- GeneralizedAssignmentGreedyLocalSearch(gap12.1)
list(solution = s$solution,
     objval   = s$objval,
     slack    = s$slack)

1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Para el problema [gap12.1](#gap12.1) las soluciones obtenidas han sido las siguientes: el resultado exacto del problema es ($1451$), el resultado obtenido por el método *Greedy* propuesto por Martello-Toth es ($1260$), que tras la heurística de *Búsqueda Local* toma el valor ($1433$).

## Conclusiones

El problema de asignación es un problema que a pesar de su simplicidad contextual, tiene grandes apliaciones en un amplio número de situaciones, es por ello que es necesario seguir estudiando el problema para buscar soluciones heurísticas eficientes y cercanas a la optimalidad. A pesar de las modelizaciones descritas en este trabajo, existen numeros ampliaciones que realizar sobre estas, tales como el aumento de la dimensionalidad del problema o la permisión de distintos valores en la demanda, al igual que ocurre con la asignación generalizada y la capacidad.

En cuanto a las modelizaciones descritas en este trabajo, estas permiten la modelización de una gran cantidad de situaciones, por lo que han sido ampliamente estudiadas en la literatura, por lo que existe una gran cantidad de heurísticas cercanas a la optimalidad, tales como las descritas en este trabajo (*Greedy* y *Búsqueda Local*) dado que cuando el tamaño del problema es inmanejable para métodos exactos, dichas heurísticas son la única manera de aproximarnos a soluciones óptimas.

## Referencias

  * [TRC13] Team, R.C., 2013. R: A language and environment for statistical computing.
  * [GP18] Sergio García Prado. Programación Entera: Heurísticas, 2018. [github.com/garciparedes/integer-programming-heuristics](https://github.com/garciparedes/integer-programming-heuristics).
  * [SA18] Jesús Sáez Aguado. Programación Entera, 2017/18. Facultad de Ciencias: Departamento de Estadística e Investigación Operativa.
  * [THBSST17] Theussl, S., Hornik, K., Buchta, C., Schwendinger, F., Schuchardt, H. and Theussl, M.S., 2017. Package ‘Rglpk’.

  